In [1]:
import functools

import numpy as np
import tensorflow as tf

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

24576/13049 [========================================================] - 0s 0us/step


In [4]:
np.set_printoptions(precision=3, suppress=True)

In [8]:
LABELS = [0, 1]
LABEL_COLUMN = 'survived'

In [12]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs)
    
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [15]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [17]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'female']
age                 : [25. 28. 47. 22. 36.]
n_siblings_spouses  : [1 0 1 0 0]
parch               : [1 0 1 0 0]
fare                : [ 30.      7.75   52.554   9.35  135.633]
class               : [b'Second' b'Third' b'First' b'Third' b'First']
deck                : [b'unknown' b'F' b'D' b'unknown' b'C']
embark_town         : [b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'n' b'y' b'y']


In [18]:
CSV_COLMUNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLMUNS)
show_batch(temp_dataset)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [24. 31. 39. 22. 35.]
n_siblings_spouses  : [0 1 1 0 0]
parch               : [3 0 5 0 0]
fare                : [19.258 18.    31.275  7.796  7.05 ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'n' b'y' b'y']


In [19]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [24. 28. 23. 37. 28.]
n_siblings_spouses  : [2 0 0 0 0]
class               : [b'Third' b'Second' b'Third' b'First' b'Third']
deck                : [b'unknown' b'E' b'unknown' b'C' b'unknown']
alone               : [b'n' b'y' b'y' b'n' b'y']


# Preprocessing the data

In [20]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path,
                            select_columns=SELECT_COLUMNS,
                            column_defaults=DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 35. 30. 47. 32.]
n_siblings_spouses  : [0. 0. 0. 0. 0.]
parch               : [1. 0. 0. 0. 0.]
fare                : [55.     7.05  10.5   38.5   56.496]


In [23]:
example_batch, labels_batch = next(iter(temp_dataset))

In [24]:
# Convert a numeric column to a vector
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [27]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print(labels.numpy())

[[ 28.5     0.      0.      7.229]
 [ 39.      0.      0.     26.   ]
 [ 58.      0.      0.    146.521]
 [ 38.      0.      0.    227.525]
 [  3.      1.      1.     26.   ]]
[0 0 1 1 1]


In [28]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
age                 : [62.   2.  14.  30.5 29. ]
n_siblings_spouses  : [0 0 0 0 1]
parch               : [0 1 0 0 0]
fare                : [10.5   12.288  7.854  8.05  27.721]
class               : [b'Second' b'Third' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [31]:
example_batch, labels_batch = next(iter(temp_dataset))

In [32]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
    
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [33]:
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES)
)

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES)
)

In [34]:
show_batch(packed_train_data)

sex                 : [b'female' b'male' b'female' b'female' b'female']
class               : [b'Second' b'Third' b'First' b'First' b'First']
deck                : [b'unknown' b'unknown' b'D' b'E' b'C']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'n' b'n' b'n' b'n']
numeric             : [[ 36.      1.      0.     26.   ]
 [ 18.      1.      1.      7.854]
 [ 23.      1.      0.    113.275]
 [ 39.      1.      1.     79.65 ]
 [ 31.      0.      2.    164.867]]


In [35]:
example_batch, labels_batch = next(iter(packed_train_data))

In [37]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


# Normalization data

In [38]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

print(MEAN)
print(STD)

[29.631  0.545  0.38  34.385]
[12.512  1.151  0.793 54.598]


In [39]:
def normalize_numeric_data(data, mean, std):
    # Center the data

    return (data-mean) / std

In [40]:
# See what you just crated.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x0000028487A86CA0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [43]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 45.   ,   0.   ,   0.   ,   8.05 ],
       [ 40.   ,   0.   ,   0.   ,   7.896],
       [ 70.   ,   1.   ,   1.   ,  71.   ],
       [ 45.   ,   1.   ,   1.   , 164.867],
       [ 58.   ,   0.   ,   0.   ,  29.7  ]], dtype=float32)>

In [45]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 1.228, -0.474, -0.479, -0.482],
       [ 0.829, -0.474, -0.479, -0.485],
       [ 3.226,  0.395,  0.782,  0.671],
       [ 1.228,  0.395,  0.782,  2.39 ],
       [ 2.267, -0.474, -0.479, -0.086]], dtype=float32)

In [46]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class': ['First', 'Second', 'Third'],
    'deck': ['A','B','C','D','E','F','G','H','I','J'],
    'embark_town': ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [47]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [48]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [49]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [50]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [51]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     1.228 -0.474
 -0.479 -0.482  1.     0.   ]


# Create model

In [52]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])